In [19]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "token_transfers"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name])
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

2025-01-15 19:07:51 [info     ] prepared 22 input batches.     filename=byblock.py lineno=78 process=44437
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain', value='op'),
                                      PartitionColumn(name='dt',
                                                      value='2024-11-18')]),
           read_from=DataLocation.GCS,
           dataset_paths={'ingestion/logs_v1': ['gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet']},
           inputs_ready=True,
           extra_marker_data={'max_block': 128146000,
                              'min_block': 128144000,
                              'num_blocks': 2000})
2025-01-15 19:07:51 [info     ] reading dataset='ingestion/logs_v1' using 1/1 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=44437

INPUT: ingestion/logs_v1

AUX VIEW: token_transfers


In [20]:
logs_view = input_datasets["ingestion/logs_v1"].create_view()

all_transfers = aux_views["token_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": logs_view,
    },
)

2025-01-15 19:07:54 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=44437
2025-01-15 19:07:54 [info     ] created table/view ingestion_logs_v1_view filename=client.py lineno=189 process=44437
2025-01-15 19:07:54 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=44437
2025-01-15 19:07:54 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=44437 template=token_transfers


In [36]:
df = ctx.client.sql(f"SELECT *, indexed_args[1], indexed_args[2], indexed_args[3] FROM {logs_view} where topic0 LIKE '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef%' and indexed_args[3] is not null LIMIT 10").df()
df

,network,chain_id,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,address,topics,data,topic0,indexed_args,chain,dt,indexed_args[1],indexed_args[2],indexed_args[3]
0,mainnet,10,1731890757,128145990,0x1dbc3f2bc6e28592c242c2da70c30d75cda987cee20f...,0x8c8097be6a11606a35153d59a846dd0563f461c67a2a...,8,38,0x416b433906b1b72fa758e166e239c43d68dc6f29,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x00000000000000000000000004f0d809f7697721387...,op,2024-11-18,0x00000000000000000000000004f0d809f76977213873...,0x000000000000000000000000ca573537505cce0d0396...,0x00000000000000000000000000000000000000000000...
1,mainnet,10,1731890759,128145991,0xc88a780a0e73dda749ad445dc0a79ea40004b8863bbe...,0x0d3b8862452f1c602e247818e4b0b17ee71da7fb67ef...,32,115,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000000000000000000000000...,op,2024-11-18,0x00000000000000000000000000000000000000000000...,0x000000000000000000000000b7659fb7387584a33716...,0x00000000000000000000000000000000000000000000...
2,mainnet,10,1731890767,128145995,0x4fdd2eb67cf6a125a756fc73d699bc50bcf67cb5b519...,0xcfa408dd1654af2fbbffdec4a3411a857118ca58d567...,11,61,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000000000000000000000000...,op,2024-11-18,0x00000000000000000000000000000000000000000000...,0x0000000000000000000000009be0b20a1eb1065b49f0...,0x00000000000000000000000000000000000000000000...
3,mainnet,10,1731890771,128145997,0x4b93310e7e2306a99be902e4f5d2482c9e558a9f315f...,0x5b7fa54141974a14be23a1fb12f301c7439439145230...,24,53,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000000000000000000000000...,op,2024-11-18,0x00000000000000000000000000000000000000000000...,0x000000000000000000000000c3cdb79d301c09764065...,0x00000000000000000000000000000000000000000000...
4,mainnet,10,1731890773,128145998,0x6b6d0b337b0c783371bf8d2af01f3d126386c82f5639...,0x820b6e0b0939c0954deb3dee3291c7aafa76204af6e4...,9,19,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000000000000000000000000...,op,2024-11-18,0x00000000000000000000000000000000000000000000...,0x0000000000000000000000008f37fdd3037b29195975...,0x00000000000000000000000000000000000000000000...
5,mainnet,10,1731890555,128145889,0xb20fd3577dbb3f6ed26a62545b38fb560b25e78a3b9d...,0xde3447431d7cb168dc4f1f2403d13e3f6292731f9524...,10,499,0x416b433906b1b72fa758e166e239c43d68dc6f29,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000000000000000000000000...,op,2024-11-18,0x00000000000000000000000000000000000000000000...,0x00000000000000000000000030a9674a5fdfc002e93d...,0x00000000000000000000000000000000000000000000...
6,mainnet,10,1731890555,128145889,0xb20fd3577dbb3f6ed26a62545b38fb560b25e78a3b9d...,0xde3447431d7cb168dc4f1f2403d13e3f6292731f9524...,10,511,0x416b433906b1b72fa758e166e239c43d68dc6f29,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x00000000000000000000000030a9674a5fdfc002e93...,op,2024-11-18,0x00000000000000000000000030a9674a5fdfc002e93d...,0x0000000000000000000000005b3fa026036ec92cb865...,0x00000000000000000000000000000000000000000000...
7,mainnet,10,1731890555,128145889,0xb20fd3577dbb3f6ed26a62545b38fb560b25e78a3b9d...,0xde3447431d7cb168dc4f1f2403d13e3f6292731f9524...,10,69,0x416b433906b1b72fa758e166e239c43d68dc6f29,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000